# Practicing Nonlinear Estimation and Control

In this unit we are going to explore more advance estimation and control on a 1-dimensional, kinematic model of the quadrotor-camera system. 

Here are some references that may be of use for this module:

+ [Extended Kalman Filter Theory](https://en.wikipedia.org/wiki/Extended_Kalman_filter)
+ [Extended Kalman Filter Example](https://home.wlu.edu/~levys/kalman_tutorial/)
+ [Extended Kalman Filter Code](https://github.com/AtsushiSakai/PythonRobotics#extended-kalman-filter-localization)
+ [PID Control Concepts](https://www.youtube.com/watch?v=UR0hOmjaHp0)
+ [PID Control Theory](https://en.wikipedia.org/wiki/PID_controller)
+ [PX4 Control Structure](https://dev.px4.io/en/flight_stack/controller_diagrams.html)
+ [PX4 Control Structure Discussion](http://discuss.px4.io/t/control-loop-architecture/3114)

## 1D Kinematic Quadrotor

Imagine your quadrotor is constrained to move in only one dimension, i.e. along the world x-axis, at some height $z$. The quadrotor moves along this constrained path by applying changes to it's pitch, $\theta$, which accelerates the system according to the equation:

$\dot{v} = -k \text{sin}(\theta)$

where your control input is $\dot{\theta}$. Note that we are only concerning ourselves the kinematics of the system, ignoring the dynamics (forces, torques).

There is some target at location $(x_T, z_T)$ that is observed by the quadrotor's downward facing camera (for now let's ingore any limitations on viewing angle). More specifically the camera can measure the angle $\gamma$ from the centerline of the camera to the target. Since the view angle depends on the pitch of the camera, $\gamma$ is a function of pitch in the form:

$\gamma = \alpha - \theta = \text{tan}^{-1}\left( \frac{x_{T}-x}{z_{T}-z}\right) - \theta$

The objective is to issue pitch rate commands, $u = \dot{\theta}_{cmd}$, in order to move the quadrotor to a position such that $\gamma \rightarrow \gamma_{des}$. For example, if $\gamma_{des} = 0$, then the objective is simply to move the quadrotor directly over the target and point downward.

This system is depicted in the below diagram:

<img src="files/kinematic_quadrotor_1d.jpg">

_Note on angle orientation:_ all angles have an orientation, i.e. they "point" from one reference line to another, in this way they all have a positive or negative sign associated with them. If the angle points in the same direction as a positive rotation about the y-axis, then the angle is positive. Conversely, angles that point in the direction of a negative y-rotation are negative

_Note on sensing:_ that the positions and velocities (i.e. $x, v, x_T, z_T, $ etc.) are not directly measurable. Angles, angular rates, and accelerations $\gamma, \theta, \dot{\theta}, \dot{v}$ are measurable but with noise.

Now we will provide the code necessary to simulate this system

In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

_HEIGHT = -1.0
_VEL_CONST = 1.0
_TIME_STEP = 0.05
_THETA_LIMIT = np.pi/4.0

## Plant Dynamics, Sensors, and Actuators

the following object contains functions for the plant dynamics, sensing of the target angle $\gamma$, and actuator for $v_{cmd}$

In [ ]:
class KinematicQuadrotor1D():
    '''Object that defines the dynamics of the simple slide-camera'''
    
    def __init__(self, x_0, z, v_0, theta_0, dtheta_0, x_T, z_T, 
                 C_vtheta=_VEL_CONST, gamma_d=0.0, theta_limit=_THETA_LIMIT):
        
        # initialize state and control variables
        self.__x = x_0
        self.__z = z # constant
        self.__v = v_0
        self.__theta = theta_0
        self.__dtheta = dtheta_0
        self.__acc = -C_vtheta*np.sin(theta_0)
        self.dtheta_cmd = 0.0
        
        # target position (hidden)
        self.__x_T = x_T
        self.__z_T = z_T
        
        # desired view angle (observed)
        self.gamma_d = gamma_d
        
        # parameters
        self.__C_vtheta = C_vtheta
        self.__theta_limit = theta_limit
        
        # sensor/actuator noise
        self.__noise_std = dict()
        self.__noise_std['gamma_s'] = 0.01
        self.__noise_std['theta_s'] = 0.001
        self.__noise_std['dtheta_s'] = 0.001
        self.__noise_std['acc_s'] = 0.005
        self.__noise_std['dtheta_a'] = 0.0001
        
        # data log
        data = dict()
        data['t'] = []
        data['dtheta_cmd'] = []
        data['dtheta_act'] = []
        data['dtheta_sens'] = []
        data['theta_act'] = []
        data['theta_sens'] = []
        data['gamma_act'] = []
        data['gamma_sens'] = []
        data['err_gamma'] = []
        data['acc_act'] = []
        data['acc_sens'] = []
        data['x_act'] = []
        data['p_act'] = []
        data['v_act'] = []
        self.data = data 
        
    def __get_theta(self):
        '''return true pitch'''
        return self.__theta
    
    def __get_position(self):
        '''return true position of quadrotor'''
        return self.__x, self.__z
    
    def __get_target_position(self):
        '''return true target position'''
        return self.__x_T, self.__z_T
    
    def __get_velocity(self):
        '''return true velocity'''
        return self.__v
    
    def __calc_gamma(self):
        '''return true target view angle'''
        return np.arctan2(self.__x_T - self.__x, self.__z_T - self.__z) - self.__theta
        
    def sense_gamma(self):
        '''sense angle from camera center line to target (includes noise)'''
        return  self.__calc_gamma() + np.random.normal(0.0, self.__noise_std['gamma_s'])
    
    def sense_theta(self):
        '''sense pitch angle (includes noise)'''
        return self.__theta + np.random.normal(0.0, self.__noise_std['theta_s'])
    
    def sense_dtheta(self):
        '''sense pitch rate (includes noise)'''
        return self.__dtheta + np.random.normal(0.0, self.__noise_std['dtheta_s'])
    
    def sense_accel(self):
        '''sense acceleration (includes noise)'''
        return self.__acc + np.random.normal(0.0, self.__noise_std['acc_s'])
        
    
    def actuate_dtheta_command(self, dtheta_cmd, dt):
        '''apply actuation to system and propagate dynamics'''
        
        # apply actuation noise
        dth = dtheta_cmd + np.random.normal(0.0, self.__noise_std['dtheta_a'])
        
        # handle theta-limited case
        th1 = self.__theta
        th2 = self.__theta + dth*dt
        th2_limited = min(self.__theta_limit, max(th2, -self.__theta_limit))
        dth_limited = (th2_limited - th1)/dt
        
        # update state and control variables
        self.dtheta_cmd = dtheta_cmd
        self.__dtheta = dth_limited
        self.__theta = th2_limited
        self.__acc = -self.__C_vtheta * np.sin(self.__theta)
        self.__v += self.__acc * dt
        self.__x += self.__v*dt 
        
    def log_data(self, t):
        '''log truth (act) and sesnor (sens) data '''
        
        self.data['t'].append(t)
        self.data['dtheta_cmd'].append(dtheta_cmd)
        self.data['dtheta_act'].append(self.__dtheta)
        self.data['theta_act'].append(self.__theta)
        self.data['acc_act'].append(self.__acc)
        self.data['gamma_act'].append(self.__calc_gamma())
        #self.data['err_gamma'].append(err_gamma)
        self.data['x_act'].append(self.__x)
        self.data['p_act'].append(self.__x_T - self.__x)
        self.data['v_act'].append(self.__v)

## Controllers

Functions for control algorithms such as proportional control, proportional-derivative control, etc, as well as any custom controllers you may wish to try

In [ ]:
def p_control(y_err, kp):
    ''' compute the actuator command based on proportional error between output and desired output
    Args:
    y_err: y_des - y where y is the output variable of the plant
    '''
    
    # TODO: write a proportional control law (hint: it is a single line, very simple equations)
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return cmd

In [ ]:
def pd_control(y_err, dy_err, kp, kd):
    '''compute the actuator command based on proportional and derivative error between output and target
    Args:
    y_err: y_des - y where y is the output variable of the plant
    y_err_prev: previous stepp y_err
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return cmd

In [ ]:
class NestedPositionController():
    
    def __init__(self, p_des, kp_p, kp_v, kp_th, kd_p, alpha_de_p):
        ''' nested controller for position, velocity, and pitch to generate pitchrate command
        Attributes:
        p_des: $p = x_t - x$ setpoint
        v_des: velocity setpoint
        theta_des: pitch setpoint
        e_p: position (x_t-x) error
        e_v: velocity error
        e_th: pitch error
        de_p: position error derivative
        alpha_de_p: smoothing factor of position error derivative
        kp_p: position proportional gain
        kp_v: velocity proportional gain
        kp_th: pitch proportional gain
        kd_p: position derivative gain
        Notes:
        - The position controller sets the velocity setpoint. The velocity controller in turn sets the 
            pitch setpoint. The pitch controller sets the pitch rate setpoint
        '''
        
        # setpoints
        self.p_des = p_des
        self.v_des = 0.0
        self.theta_des = 0.0
        
        # errors
        self.e_p = 0.0
        self.e_v = 0.0
        self.e_th = 0.0
        self.de_p =0.0
        self.alpha_de_p = alpha_de_p
        
        # gains
        self.kp_p = kp_p
        self.kp_v = kp_v
        self.kp_th = kp_th
        self.kd_p = kd_p
        
        # data log
        self.data = dict()
        self.data['t'] = []
        self.data['p_des'] = []
        self.data['v_des'] = []
        self.data['theta_des'] = []
        
    def compute_dtheta_command(self, x_est, dt):
        '''generate dtheta control signal'''
        
        # YOUR CODE HERE
        raise NotImplementedError()

        return self.dtheta_cmd
    
    def position_controller(self, p_est, dt):
        '''get velocity setpoint from position controller
        Notes:
        - The negative sign in the setpoint calc is due to the inversion between position and velocity when using p=x_T-x.
            i.e. positive p (x_T > x), creates negative error (assumin p_des=0), but require positive velocity to correct
        '''
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        
    def velocity_controller(self, v_est, dt):
        '''get pitch setpoint from velocity controller
        Notes:
        - The negative sign in the setpoint calc is due to the inversion between velocity and pitch.
            i.e. positive velocity error (v_des > v) requires a pitch down to accelerate forward and rectify
        - Theta is limited, so no point and commanding greater than some limit
        '''
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def pitch_controller(self, th_est, dt):
        '''get pitch rate setpoint from pitch controller
        Notes:
        - No negative sign on this controller. i.e. positive pitch error (theta_des > theta) requires pitch up to rectify
        '''
        # YOUR CODE HERE
        raise NotImplementedError()
        
    
    def log_data(self, t):
        self.data['t'].append(t)
        self.data['p_des'].append(self.p_des)
        self.data['v_des'].append(self.v_des)
        self.data['theta_des'].append(self.theta_des)
        

## Estimators

State estimation algorithms. See references:
    - https://en.wikipedia.org/wiki/Extended_Kalman_filter
    - http://web.mit.edu/kirtley/kirtley/binlustuff/literature/control/Kalman%20filter.pdf
    - https://home.wlu.edu/~levys/kalman_tutorial/

In [ ]:
class EKF_KinematicQuadrotor1D():
    '''Extended Kalman Filter for estimating relative position (p), velocity (v) and pitch (theta) of 
            1-D kinematic quadrotor system
     Notation (see https://en.wikipedia.org/wiki/Extended_Kalman_filter):
    - x[0] = $p = x_{T} - x$: position of target relative to position of quadrotor
    - x[1] = $v$: velocity of quadrotor in x direction
    - x[2] = $\theta$: pitch of quadrotor
    - u[0] = $\dot{\theta}$: pitch rate of quadrotor (commanded or measured....?)
    - z[0] = $\gamma$: measured view angle from camera centerline to target
    - z[1] = $a$: measured linear acceleration of quadrotor
    - z[2] = $\theta$: measure pitch of quadrotor
    - C_vtheta = constant that maps pitch to linear acceleration: $\dot{v} = -C_{v\theta} \text{sin}(\theta)$
    - h_z = $z_{T} - z$ height of target relative to height of quadrotor (assumed constant and known)
    - x_k1_k1 = \hat{x}_{k-1|k-1} estimate of state at step k-1 based on step k-1
    - x_k_k1 = \hat{x}_{k|k-1} predicted state at step k based on step k-1
    - F_k = F_{k} jacobian of dynamics f with respect to state x, evaluated at x_{k|k-1}, u_{k}
    - P_k_k1 = P_{k|k-1} predicted covariance at step k based on step k-1
    - P_k1_k1 = P_{k|k-1} estimated covariance at step k-1 based on step k-1
    '''
    
    def __init__(self, C_vtheta, h_z, p_0, v_0, theta_0):
        
        # state predictions and estimates
        self.x_k_k = np.array([p_0, v_0, theta_0])
        self.x_k_k1 = np.array([p_0, v_0, theta_0])
        self.x_k1_k1 = np.array([p_0, v_0, theta_0])
        
        # covariance predictions and estimates
        self.P_k_k1 = 0.0*np.eye(3)
        self.P_k_k = 0.0*np.eye(3)
        self.P_k1_k1 = 0.0*np.eye(3)
        
        # measurement and residual
        self.y_k = np.array(3*[0.0])
        self.z_k = np.array(3*[0.0])
        
        # actuator (process) covariance
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # observation (sensor) covariance
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # known constants and parameters
        self.C_vtheta = C_vtheta
        self.h_z = h_z
        
        # data log
        self.data = dict()
        self.data['t'] = []
        self.data['p_pred'] = []
        self.data['p_est'] = []
        self.data['v_pred'] = []
        self.data['v_est'] = []
        self.data['theta_pred'] = []
        self.data['theta_est'] = []
        self.data['gamma_sens'] = []
        self.data['acc_sens'] = []
        self.data['theta_sens'] = []
        self.data['y_gamma'] = []
        self.data['y_acc'] = []
        self.data['y_theta'] = []
    
    
    def predict(self, u_k, dt):
        ''' prediction step of k based on step k-1
        '''
        
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def update(self, z_k):
        ''' estimation update for step k based on step k
        Args:
        - z_k: measurements taken at step k
        '''
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def log_data(self, t):
        '''log estimator data'''
        
        self.data['t'].append(t)
        self.data['p_pred'].append(self.x_k_k1[0])
        self.data['v_pred'].append(self.x_k_k1[1])
        self.data['theta_pred'].append(self.x_k_k1[2])
        self.data['p_est'].append(self.x_k_k[0])
        self.data['v_est'].append(self.x_k_k[1])
        self.data['theta_est'].append(self.x_k_k[2])
        self.data['gamma_sens'].append(self.z_k[0])
        self.data['acc_sens'].append(self.z_k[1])
        self.data['theta_sens'].append(self.z_k[2])
        self.data['y_gamma'].append(self.y_k[0])
        self.data['y_acc'].append(self.y_k[1])
        self.data['y_theta'].append(self.y_k[2])
    

## Simulation Script

below is a script for testing various controllers for the `SimpleSlideCamera` plant as well as plotting the results.

In [ ]:
# Simulation Inputs
dt = 0.01
t_final = 50.0

# intial conditions (position, velocity and targe position)
x_0 = 0.0
z = -1.0
v_0 = 0.0
theta_0 = 0.0
dtheta_0 = 0.0
x_T = 1.0
z_T = 0.0

# create SimpleSlideCamera with initial conditions
quad = KinematicQuadrotor1D(x_0, z, v_0, theta_0, dtheta_0, x_T, z_T, C_vtheta=_VEL_CONST)

# create controller
ctrl = NestedPositionController(p_des=0.0, kp_p=1.0, kp_v=0.2, kp_th=0.5, kd_p = 4.0, alpha_de_p = 0.3)

# initialize EKF state estimator
ekf = EKF_KinematicQuadrotor1D(C_vtheta=_VEL_CONST, h_z=z_T-z, p_0=x_T-x_0, v_0=v_0, theta_0=theta_0)

# initialize data storage
t = 0.0
dtheta_cmd = 0.0
while t < t_final:
    
    # ESTIMATOR: predict next state
    ekf.predict([dtheta_cmd], dt)
    
    # ESTIMATOR: update state estimate based on sensor data
    z_k = np.array([quad.sense_gamma(), quad.sense_accel(), quad.sense_theta()])
    ekf.update(z_k)
    
    # CONTROLLER: call theta control algoritm
    dtheta_cmd = ctrl.compute_dtheta_command(ekf.x_k_k, dt)
                                    
    # initilization delay to allow filters to "come up to speed"
    if t/dt < 10.0:
        dtheta_cmd = 0.0
    
    # store data
    quad.log_data(t)
    ekf.log_data(t)
    ctrl.log_data(t)
    
    # ACTUATOR: send velocity command to plant
    quad.actuate_dtheta_command(dtheta_cmd, dt)
    
    # increment time step
    t += dt
    #err_pos_prev1 = err_pos
    
    

## Analysis

visualize the results of you simulation to analyze performance of estimator and controller

In [ ]:
# plot size
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
# position
h_p_act, = plt.plot(quad.data['t'], quad.data['p_act'], label='truth')
h_p_est, = plt.plot(ekf.data['t'], ekf.data['p_est'], label='estimate')
h_p_des, = plt.plot(ctrl.data['t'], ctrl.data['p_des'], label='setpoint')
#h_p_pred, = plt.plot(ekf.data['t'], ekf.data['p_pred'], label='prediction')
plt.legend(handles=[h_p_act, h_p_est, h_p_des])
plt.title('Relative Position (x_T - x)', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('x-pos [m]')
plt.show()

In [ ]:
# velocity
h_v_act, = plt.plot(quad.data['t'], quad.data['v_act'], label='truth')
h_v_est, = plt.plot(ekf.data['t'], ekf.data['v_est'], label='estimate')
#h_v_pred, = plt.plot(ekf.data['t'], ekf.data['v_pred'], label='prediction')
h_v_des, = plt.plot(ctrl.data['t'], ctrl.data['v_des'], label='setpoint')
plt.legend(handles=[h_v_act, h_v_est, h_v_des])
plt.title('Velocity', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('x-vel [m/s]')
plt.show()

In [ ]:
# pitch (theta)
h_th_act, = plt.plot(quad.data['t'], quad.data['theta_act'], label='truth')
h_th_est, = plt.plot(ekf.data['t'], ekf.data['theta_est'], label='estimate')
h_th_des, = plt.plot(ctrl.data['t'], ctrl.data['theta_des'], label='setpoint')
h_th_sens, = plt.plot(ekf.data['t'], ekf.data['theta_sens'], label='measurement', linestyle="",marker=".")
# h_th_pred, = plt.plot(ekf.data['t'], ekf.data['theta_pred'], label='prediction')
plt.legend(handles=[h_v_act, h_v_est, h_th_des, h_th_sens])
plt.title('Pitch', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('pitch [rad]')
plt.show()

In [ ]:
# pitch rate command
h_dth_act, = plt.plot(quad.data['t'], quad.data['dtheta_act'], label='truth')
h_dth_cmd, = plt.plot(quad.data['t'], quad.data['dtheta_cmd'], label='command')
plt.legend(handles=[h_dth_act, h_dth_cmd])
plt.title('Pitch Rate', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('pitch rate [rad/s]')
plt.show()

In [ ]:
# target view angle (gamma)
h_gam_act, = plt.plot(quad.data['t'], quad.data['gamma_act'], label='truth')
h_gam_sens, = plt.plot(ekf.data['t'], ekf.data['gamma_sens'], label='measurement', linestyle="",marker=".")
plt.legend(handles=[h_gam_act, h_gam_sens])
plt.title('Target View Angle (gamma)', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('View Angle [rad]')
plt.show()

In [ ]:
# Acceleration
h_acc_act, = plt.plot(quad.data['t'], quad.data['acc_act'], label='truth')
h_acc_sens, = plt.plot(ekf.data['t'], ekf.data['acc_sens'], label='measurement', linestyle="",marker=".")
plt.legend(handles=[h_acc_act, h_acc_sens])
plt.title('Acceleration', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('Acceleration [rad]')
plt.show()

In [ ]:
# residual
h_y_gamma, = plt.plot(ekf.data['t'], ekf.data['y_gamma'], label='view angle [rad]')
h_y_acc, = plt.plot(ekf.data['t'], ekf.data['y_acc'], label='acceleration [m/s/s]')
h_y_theta, = plt.plot(ekf.data['t'], ekf.data['y_theta'], label='theta [rad]')
plt.legend(handles=[h_y_gamma, h_y_acc, h_y_theta])
# plt.legend(handles=[h_y_gamma])
plt.title('Measurment Residual', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('residual')
plt.show()

## Questions

1. Can you design a controller that is capable of converging the gamma error to zero?